## generate narrative from resource

### import modules

In [68]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
#from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
#from fhirclient.r4models import bundle as B
from fhirclient.r4models import narrative as N
#import fhirclient.models.identifier as I
#import fhirclient.r4models.identifier as I
#import fhirclient.r4models.coding as C
#import fhirclient.r4models.codeableconcept as CC
#import fhirclient.r4models.fhirdate as D
#import fhirclient.r4models.extension as X
#import fhirclient.r4models.contactdetail as CD
#import fhirclient.r4models.fhirreference as FR
from json import dumps, loads, load, JSONDecodeError
from requests import get, post, put
#import os
from pathlib import Path
#from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
#from pprint import pprint
#from collections import namedtuple
#from pandas import *
from datetime import datetime, date
from jinja2 import Environment, FileSystemLoader, select_autoescape
from stringcase import snakecase, titlecase
#from itertools import zip_longest
from commonmark import commonmark
from htmlmin import minify

### Define variables

#### get CapStatement as json

In [69]:
fname = input('enter the CapStatment file name:')

enter the CapStatment file name: pdex-server.json


In [71]:
path = Path.cwd() / 'cs_source_file'/ fname
cs_json = path.read_text()
cs_py = CS.CapabilityStatement(loads(cs_json))
print(cs_py.id)
# sp_map = {sp.code:sp.type for sp in df_sp.itertuples(index=True)}
# pname_map = {p.Profile:p.Name for p in df_profiles.itertuples(index=True)}

# this is wrong if for most IGs should be supportedProfiles not Profiles
pname_map = {r.profile:titlecase(r.profile.split('/')[-1]) for r in cs_py.rest[0].resource if r.profile}
#add in supported Profiles too
spname_map = {sp:titlecase(sp.split('/')[-1]) for \
              r in cs_py.rest[0].resource if r.supportedProfile for sp in r.supportedProfile}
#flatten_matrix = [val for sublist in matrix for val in sublist] 

# Adding elements from dict2 to dict1
pname_map.update(spname_map)

sp_map = None

print(pname_map)

pdex-server
{'http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance': 'Us Core Allergyintolerance', 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careplan': 'Us Core Careplan', 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-careteam': 'Us Core Careteam', 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-condition': 'Us Core Condition', 'http://hl7.org/fhir/us/davinci-hrex/StructureDefinition/hrex-coverage': 'Hrex Coverage', 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/pdex-device': 'Pdex Device', 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-implantable-device': 'Us Core Implantable Device', 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-note': 'Us Core Diagnosticreport Note', 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-diagnosticreport-lab': 'Us Core Diagnosticreport Lab', 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-documentreference': 'Us Core Documentreference

### get pathmap from package file
- Get spec_internal from package a json file which includes canonical to local relative page links

In [72]:
ig_package = input("Get package path to download:")

Get package path to download: http://build.fhir.org/ig/HL7/davinci-epdx/package.tgz


In [73]:
r = get(ig_package, verify=False)
r.status_code
#write to file
path = Path.cwd() /  'cs_source_file' / ig_package.split('/')[-1]
path.write_bytes(r.content)

86288

In [74]:
import tarfile

def get_si(tar_file):
    print(path)
    with tarfile.open(tar_file, mode='r') as tf:
        #pprint(tf.getnames())
        f = tf.extractfile('package/other/spec.internals')
        r = f.read()
        return(loads(r))


si = get_si(path)
       
path_map = si['paths']
path_map

C:\Users\Eric\Documents\Python\MyNotebooks\CapStatement\cs_source_file\package.tgz


{'http://hl7.org/fhir/us/davinci-pdex/ImplementationGuide/hl7.fhir.us.davinci-pdex|1.0.0': '1.0.0/ImplementationGuide-hl7.fhir.us.davinci-pdex.html',
 'http://hl7.org/fhir/us/davinci-pdex/ImplementationGuide/hl7.fhir.us.davinci-pdex': 'ImplementationGuide-hl7.fhir.us.davinci-pdex.html',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/hrex-coverage|1.0.0': '1.0.0/StructureDefinition-hrex-coverage.html',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/hrex-coverage': 'StructureDefinition-hrex-coverage.html',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/pdex-device|1.0.0': '1.0.0/StructureDefinition-pdex-device.html',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/pdex-device': 'StructureDefinition-pdex-device.html',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/pdex-medicationdispense|1.0.0': '1.0.0/StructureDefinition-pdex-medicationdispense.html',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/pdex-medicationdispense': 

### Get URL:Name/Title mapping from package file

In [75]:
from json import JSONDecodeError
def get_name_map(tar_file):
    print(path)
    name_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            f = tf.extractfile(p_file)
            r = f.read()
            #print(type(r))
            try:
                r_dict = loads(r)
            except JSONDecodeError: # not a json file
                continue
            try:
                name_map[r_dict['url']] = r_dict['title']
                print(r_dict['title'])
            except KeyError:
                try:
                    name_map[r_dict['url']] = r_dict['name']
                    print(r_dict['name'])
                except KeyError: # not a conf resource
                    print(f'****{p_file}is not a conf resource***')
        return(name_map)


name_map = get_name_map(path)
name_map       

C:\Users\Eric\Documents\Python\MyNotebooks\CapStatement\cs_source_file\package.tgz
Da Vinci Payer Data exchange
Da Vinci Payer Data exchange
Da Vinci Payer Data exchange
HRex Coverage Profile
PDex Device
PDex MedicationDispense Profile
PDex Provenance
An attribute to express the refill number of a prescription
An attribute to describe the data source a resource was constructed from
FDA National Drug Code (NDC)
Provenance Agent Type
Payer source of data
Identifier Type
Provenance Roles
Provenance Payer Data Source Format
****package/example/Patient-1.jsonis not a conf resource***
****package/example/MedicationDispense-1000001.jsonis not a conf resource***
****package/example/Provenance-1000001.jsonis not a conf resource***
****package/example/Provenance-1000002.jsonis not a conf resource***
****package/example/Provenance-1000003.jsonis not a conf resource***
****package/example/Provenance-1000004.jsonis not a conf resource***
****package/example/Provenance-1000005.jsonis not a conf reso

{'https://build.fhir.org/ig/HL7/davinci-epdx/': 'Da Vinci Payer Data exchange',
 'http://hl7.org/fhir/us/davinci-pdex/ImplementationGuide/hl7.fhir.us.davinci-pdex': 'Da Vinci Payer Data exchange',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/hrex-coverage': 'HRex Coverage Profile',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/pdex-device': 'PDex Device',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/pdex-medicationdispense': 'PDex MedicationDispense Profile',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/pdex-provenance': 'PDex Provenance',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/DispenseRefill': 'An attribute to express the refill number of a prescription',
 'http://hl7.org/fhir/us/davinci-pdex/StructureDefinition/ProvenanceSourceFrom': 'An attribute to describe the data source a resource was constructed from',
 'http://hl7.org/fhir/us/davinci-pdex/ValueSet/FDANationalDrugCode': 'FDA National Drug Code (NDC)',
 'http:

### Get SearchParameter parameter name: type mapping from package file

In [76]:
def get_sp_map(tar_file):
    print(path)
    sp_map = {}
    with tarfile.open(tar_file, mode='r') as tf:
        for p_file in tf.getnames():
            #print(p_file)
            if 'searchparameter' in p_file.lower():
                print(f'**********{p_file}')
                f = tf.extractfile(p_file)
                r = f.read()
                #print(type(r))
                try:
                    r_dict = loads(r)
                except JSONDecodeError: # not a json file
                    continue
                try:
                    sp_map[r_dict['code']] = r_dict['type']
                    print(r_dict['type'])
                except KeyError:
                    print(f'{p_file} has no type')
        return(sp_map)


sp_map = get_sp_map(path)
sp_map       

C:\Users\Eric\Documents\Python\MyNotebooks\CapStatement\cs_source_file\package.tgz
**********package/SearchParameter-pdex-medicationdispense-patient.json
reference
**********package/SearchParameter-pdex-medicationdispense-status.json
token


{'patient': 'reference', 'status': 'token'}

### render using Jinja2 templates

In [77]:
in_path = ''
in_file = 'R4capabilitystatement-server.j2'

def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = in_path),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'],),
    trim_blocks = True,
    lstrip_blocks = True,
    )

env.filters['markdown'] = markdown

template = env.get_template(in_file)

rendered = template.render(cs=cs_py, path_map=path_map, pname_map=name_map, sp_map=sp_map )

### Minify the xhtml

In [78]:
def x_minify(xhtml):
    h_min=minify(xhtml, remove_optional_attribute_quotes=False, remove_comments=True)
    x_min = h_min.replace('<br>','<br />')
    x_min = x_min.replace('<hr>','<hr />')
    return x_min

mini = x_minify(rendered)
#print(type(mini))
#print(mini)
#display(HTML(rendered))
display(HTML(mini))

'''
#======== write to temp file to debug =======
path = Path.cwd() / 'debug' / 'narrative_pre.xhtml'
path.write_text(rendered, encoding="utf-8")
path = Path.cwd() / 'debug' / 'narrative_mini_pre.xhtml'
path.write_text(mini, encoding="utf-8")
#===================================================
'''
narr = N.Narrative()
narr.status = 'generated'
narr.div = mini
cs_py.text = narr

Resource Type,Supported Profiles,Supported Searches,Supported _includes,Supported _revincludes,Supported Operations
AllergyIntolerance,,patient patient,,Provenance:target,
CarePlan,,"category, date, patient, status patient+category+status+date, patient+category+status, patient+category, patient+category+date",,Provenance:target,
CareTeam,,"patient, status patient+status",,Provenance:target,
Condition,,"category, clinical-status, patient, onset-date, code patient+onset-date, patient+category, patient+clinical-status, patient+code",,Provenance:target,
Coverage,,_id,,Provenance,
Device,"PDex Device,","patient, type patient+type",,Provenance:target,
DiagnosticReport,",","status, patient, category, code, date patient+category+date, patient+status, patient+code+date, patient+code, patient+category",,Provenance:target,
DocumentReference,,"_id, status, patient, category, type, date, period patient+type+period, patient+type, patient+category+date, patient+status, patient+category",,Provenance:target,docref
Encounter,,"_id, class, date, identifier, patient, status, type class+patient, patient+status, patient+type, date+patient",,Provenance:target,
Goal,,"lifecycle-status, patient, target-date patient+lifecycle-status, patient+target-date",,Provenance:target,


### TODO: add validation step

In [79]:
def validate(r):
    #fhir_test_server = 'http://fhirtest.uhn.ca/baseDstu3'
    #fhir_test_server = 'http://test.fhir.org/r3'
    #fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server ='http://wildfhir4.aegis.net/fhir4-0-0'
    #fhir_test_server ='http://wildfhir4.aegis.net/fhir4-0-0'
    fhir_test_server ='http://hapi.fhir.org/baseR4'
  

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    r = post(f'{fhir_test_server}/CapabilityStatement/$validate', headers = headers, data = dumps(cs_py.as_json()) )   # return r.status_code
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

validate(cs_py)

ERROR,[],Failed to call access method: org.springframework.dao.InvalidDataAccessApiUsageException: Can't fetch resource type: SearchParameter; nested exception is java.lang.IllegalArgumentException: Can't fetch resource type: SearchParameter


### Save to local dir

In [80]:
# save to file
ig_source_path = ''
print('...........saving to file............')
#save in ig_source folder
path = Path.cwd() / ig_source_path / 'resources' / f'capabilitystatement-{cs_py.id.lower()}.json'
path.write_text(dumps(cs_py.as_json(), indent=4))

...........saving to file............


221589